MAIN STRATEGY - MEAN REVERSION USING BOLLINGER BANDS

In [ ]:
# MAIN IDEA:
# region imports
from AlgorithmImports import * # import necessary libraries and modules
# endregion

class MultipleForexAssets(QCAlgorithm):

    def initialize(self):
        self.set_start_date(2020,6,1) # set start date for backtesting
        self.set_cash(100000) # set initial capital

        # forex pairs to trade
        self.forex_pairs = ["EURUSD", "USDJPY", "GBPUSD", "USDCHF", "AUDJPY"]

        self.bb_indicators = {} # dictionary to store Bollinger Bands indicators for each pair
        self.lookback_period = 50 # in hours
        self.std_dev_multiplier = 3 # multiplier for standard deviation
        self.stop_loss_multiplier = 4 # # multiplier for setting stop loss
        self.invest_percentage_per_pair = 0.2 # # percentage of capital to invest per pair

        # initialize Bollinger Bands indicators for each pair
        for pair in self.forex_pairs:
            self.add_forex(pair, Resolution.HOUR, Market.OANDA)
            self.bb_indicators[pair] = self.BB(pair, self.lookback_period, self.std_dev_multiplier, MovingAverageType.SIMPLE, Resolution.HOUR)

        self.set_brokerage_model(BrokerageName.OANDA_BROKERAGE) # set brokerage model


    def on_data(self, data: Slice):
        # if not invested in any pair, invest a fixed percentage of capital in each pair
        if not self.portfolio.invested:
            for pair in self.forex_pairs:
                self.set_holdings(pair, self.invest_percentage_per_pair)

        # loop through each forex pair
        for pair in self.forex_pairs:
            pair_bb = self.bb_indicators[pair]
            if not pair_bb.is_ready:
                continue

            # get Bollinger Bands values
            lower_band = pair_bb.lower_band.current.value
            upper_band = pair_bb.upper_band.current.value
            stop_loss_price = pair_bb.middle_band.current.value - self.stop_loss_multiplier * pair_bb.standard_deviation.current.value
            current_price = data[pair].close if pair in data else None

            if current_price is None:
                continue

            # strategy logic based on Bollinger Bands
            if current_price <= stop_loss_price: # if current price hits stop loss, liquidate the position
                self.liquidate(pair)
                self.debug(f"Stop loss activated for currency pair {pair}. Stop loss price: {stop_loss_price} Sell price: {current_price}")
            elif current_price <= lower_band: # if current price crosses below lower band, buy the pair
                self.set_holdings(pair, self.invest_percentage_per_pair)
                self.debug(f"Price above upper band for currency pair {pair}. Upper band: {upper_band} Buy price: {current_price}")
            elif current_price >= upper_band: # if current price crosses above upper band, sell the pair
                self.liquidate(pair)
                self.debug(f"Price below lower band for currency pair {pair}. Lower band: {lower_band} Sell price: {current_price}")

ALTERNATIVE STRATEGY THAT INCLUDES ATTEMPT TO USE SCORING MECHANISM ON TRADING IN THE STOCK MARKET

In [ ]:
# ALTERNATIVE IDEA:
# region imports
from AlgorithmImports import *
import numpy as np
# endregion

class UglyRedOrangeChicken(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2019, 1, 1)
        self.SetEndDate(2023, 4, 1)
        self.SetCash(100000)

        self.spy = self.AddEquity("SPY", Resolution.DAILY)
        self.bnd = self.AddEquity("BND", Resolution.DAILY)

        # As a baseline, we look at monthly average of indicators to match our daily trading frequency
        self.spy_momp = self.MOMP(self.spy.Symbol, 20)
        self.spy_alpha = self.A(self.spy.Symbol, self.bnd.Symbol, 20)
        self.bnd_momp = self.MOMP(self.bnd.Symbol, 20)

        # We focus on US stocks in the tech sector, selecting 6 large companies
        tech_mkt_stocks = ['AAPL', 'NVDA', 'IBM', 'INTC', 'AMD', 'TSLA']
        self.tech_mkt = {}
        for t in tech_mkt_stocks:
            profile = {}
            profile['S'] = self.AddEquity(t, Resolution.DAILY)
            symb = profile['S'].Symbol
            profile['Symb'] = symb
            profile['Alpha'] = self.A(symb, self.bnd.Symbol, 20)
            profile['Beta'] = self.B(symb, self.spy.Symbol, 20)
            profile['Blng5'] = self.BB(symb, 5, 1)
            profile['Blng20'] = self.BB(symb, 20, 1)
            profile['Blng100'] = self.BB(symb, 100, 2)
            profile['Momp'] = self.MOMP(symb, 20)
            profile['Sharpe'] = self.SR(symb, 20, 0.016)
            self.tech_mkt[t] = profile
        self.n_stock = len(self.tech_mkt)

        self.SetWarmUp(20)
        self.init_invested = False
        self.score_hist = []

        # Trading parameters
        self.balance_thres = 0.1
        self.stop_factor = 0.6
        self.reconf_factor = 0.2

    def OnData(self, data: Slice):
        if not self.IsWarmingUp:
            if not self.init_invested:
                # Initial holdings according to our baseline
                self.SetHoldings("SPY", 0.15)
                self.SetHoldings("BND", 0.2)
                for t in self.tech_mkt.keys():
                    self.SetHoldings(t, 0.55 / self.n_stock)
                self.init_invested = True
                return

            cash = self.portfolio.cash
            total_value = self.portfolio.total_portfolio_value

            # Plotting
            for e in [self.spy, self.bnd]:
                sym = e.symbol
                self.Plot("Quantity", sym, self.portfolio[sym].quantity)
                self.Plot("Value", sym, self.portfolio[sym].holdings_value)
            for e in self.tech_mkt.keys():
                self.Plot("Quantity", e, self.portfolio[e].quantity)
                self.Plot("Value", e, self.portfolio[e].holdings_value)
            self.Plot("Value", "TOTAL", total_value)
            self.Plot("Value", "Cash", cash)

            self.Plot("MOMP", "SPY", self.spy_momp.Current.Value)
            self.Plot("MOMP", "BND", self.bnd_momp.Current.Value)
            for e, p in self.tech_mkt.items():
                self.Plot("MOMP", e, p['Momp'].Current.Value)

            self.Plot("Alpha", "SPY", self.spy_alpha.Current.Value)
            for e, p in self.tech_mkt.items():
                self.Plot("Alpha", e, p['Alpha'].Current.Value)

            # Rebalancing on every Monday (on Friday data)
            if self.time.weekday() == 4:

                # Compute the bias into bond market rather than stocks, due to less optimal stock performance
                bnd_bias = 0
                if len(self.score_hist) > 0:
                    tech_avg_score = np.average(self.score_hist)
                    self.score_hist = []
                    x = tech_avg_score - (-15)
                    bnd_bias = 2 / (1 + np.exp(0.03 * x)) - 1
                    # self.Debug(str(tech_avg_score) + " " + str(bnd_bias))

                # Baseline 15% into S&P 500, 20% Bond market, 10% Cash, 55% Tech stocks
                # Adjust them based on bnd_bias, which is derived from the stock market scores
                for e, share in [(self.spy, 0.15 - 0.05 * bnd_bias), (self.bnd, 0.2 + 0.15 * bnd_bias)]:
                    tgt_qty = int(total_value * share / e.price)
                    real_qty = self.portfolio[e.symbol].quantity
                    trade_qty = int(tgt_qty - real_qty)
                    if np.abs(trade_qty) > self.balance_thres * real_qty:
                        self.market_order(e.symbol, trade_qty)

                target_stock_value = (0.55 - 0.15 * bnd_bias) * total_value
                curr_stock_value = 0
                for e in self.tech_mkt.keys():
                    curr_stock_value += self.portfolio[e].holdings_value
                if curr_stock_value == 0:
                    return
                size_factor = target_stock_value / curr_stock_value

                for e in self.tech_mkt.keys():
                    tgt_qty = np.floor(self.portfolio[e].holdings_value * size_factor / self.portfolio[e].price)
                    real_qty = self.portfolio[e].quantity
                    trade_qty = int(tgt_qty - real_qty)
                    if np.abs(trade_qty) > self.balance_thres * real_qty:
                        self.market_order(e, trade_qty)

            else:

                scores = {}
                avg_score = 0
                tech_mkt_value = 0
                for e, p in self.tech_mkt.items():
                    curr_qty = self.portfolio[e].quantity
                    curr_price = self.portfolio[e].price

                    beta = p['Beta'].Current.Value
                    mva5 = p['Blng5'].MiddleBand.Current.Value
                    mva20 = p['Blng20'].MiddleBand.Current.Value
                    mva100 = p['Blng100'].MiddleBand.Current.Value
                    high_stop = p['Blng5'].UpperBand.Current.Value
                    low_stop = p['Blng5'].LowerBand.Current.Value
                    momp = p['Momp'].Current.Value
                    sr = p['Sharpe'].Current.Value

                    stop_sell_qty = np.floor(curr_qty * self.stop_factor)

                    mv_crossover = np.sign(mva20 - mva100)
                    # Scoring formula
                    score = sr * 10 + momp + mv_crossover * 6 - beta * 3
                    scores[e] = score
                    self.Plot("MetricScore", e, score)

                    avg_score += self.portfolio[sym].holdings_value * score
                    tech_mkt_value += self.portfolio[sym].holdings_value

                # Reconfigure the assets based on the scores
                # We buy more the highest scored stock, sell the same value of lowest scored stock, if the score difference is large
                scores_sorted = sorted(scores.items(), key = lambda x: -x[1])
                score_diff = scores_sorted[0][1] - scores_sorted[-1][1]
                if score_diff > 15:
                    value_amt = min(cash, 0.1 * total_value) * self.reconf_factor
                    e_to_buy = scores_sorted[0][0]
                    e_to_sell = scores_sorted[-1][0]
                    buy_qty = value_amt / self.portfolio[e_to_buy].price
                    sell_qty = value_amt / self.portfolio[e_to_sell].price
                    if buy_qty > 5 and sell_qty > 5 and sell_qty < self.portfolio[e_to_sell].quantity:
                        self.market_order(e_to_buy, np.floor(buy_qty))
                        self.market_order(e_to_sell, -np.floor(sell_qty))

                avg_score /= tech_mkt_value
                self.score_hist.append(avg_score)

    def on_end_of_algorithm(self):
        self.liquidate()